# Projet 5 : Entraînement du Modèle de Prédiction de Démission

Ce notebook reprend la logique du Projet 4 pour nettoyer les données, créer des features, entraîner des modèles et sauvegarder le meilleur modèle pour le déploiement.

In [1]:
import warnings

import joblib
import pandas as pd
import plotly.express as px
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
)
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler

# Configuration
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

## 2. Prétraitement et Fusion
Nous nettoyons les identifiants et fusionnons les tables pour obtenir un dataset unique.

## 3. Ingénierie des Fonctionnalités (Feature Engineering)
Création des variables explicatives (ratios) et encodage des données catégorielles.

In [2]:
print("--- Ingénierie des Fonctionnalités ---")

df = pd.read_csv("master_dataset.csv")

# Nettoyage des chaînes de caractères (ex: '11 %' -> 11.0)
if df['augmentation_salaire_precedente'].dtype == 'object':
    df['augmentation_salaire_precedente'] = df['augmentation_salaire_precedente'].astype(str).str.replace(' %', '').astype(float)

# Encodage Binaire (Oui/Non -> 1/0)
binary_map = {'Oui': 1, 'Non': 0}
cols_binaires = ['heure_supplementaires']
for col in cols_binaires:
    if col in df.columns and df[col].dtype == 'object':
        df[col] = df[col].map(binary_map)

# Encodage Ordinal
map_deplacement = {'Aucun': 0, 'Occasionnel': 1, 'Frequent': 2}
if 'frequence_deplacement' in df.columns and df['frequence_deplacement'].dtype == 'object':
    df['frequence_deplacement'] = df['frequence_deplacement'].map(map_deplacement)

# Création de Ratios Métier
# Remplacement des 0 par 1 pour éviter les divisions par zéro
df['ratio_stagnation'] = df['annees_dans_le_poste_actuel'] / df['annees_dans_l_entreprise'].replace(0, 1)
df['revenu_par_annee_exp'] = df['revenu_mensuel'] / df['annee_experience_totale'].replace(0, 1)

# Définition de la Cible (Target)
y = df['a_quitte_l_entreprise'].map({'Oui': 1, 'Non': 0})

# Encodage One-Hot pour les autres variables catégorielles
X_raw = df.drop(columns=['a_quitte_l_entreprise'])
X_encoded = pd.get_dummies(X_raw, drop_first=True)

# Suppression de l'identifiant (non prédictif)
X = X_encoded.drop(columns=['id_employee'])

print("Feature Engineering terminé.")

--- Ingénierie des Fonctionnalités ---
Feature Engineering terminé.


## 4. Sélection des Features Essentielles
Nous ne conservons que les 10 variables les plus pertinentes identifiées lors de l'analyse exploratoire.

In [3]:
features_to_analyze = [
    'revenu_mensuel',
    'age',
    'distance_domicile_travail',
    'satisfaction_employee_environnement',
    'heure_supplementaires',
    'annees_depuis_la_derniere_promotion',
    'satisfaction_employee_equilibre_pro_perso',
    'nombre_participation_pee',
    'ratio_stagnation',
    'revenu_par_annee_exp'
]

# Vérification de la présence des colonnes
available_features = [f for f in features_to_analyze if f in X.columns]
if len(available_features) < len(features_to_analyze):
    print("Attention : Certaines features essentielles sont manquantes.")

X_essential = X[available_features]
print(f"Entraînement sur {X_essential.shape[1]} features essentielles.")

Entraînement sur 10 features essentielles.


## 5. Entraînement des Modèles (Baseline)
Nous comparons un Dummy Classifier, une Régression Logistique et un Random Forest.

In [4]:
def evaluer_modele(model, X_test, y_test, model_name, plot_plotly=False):
    print(f"\n{'='*40}")
    print(f"ÉVALUATION : {model_name}")
    print(f"{'='*40}")

    # 1. Prédictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # 2. Rapport de classification (Precision, Recall, F1)
    print(classification_report(y_test, y_pred))

    # 3. Score ROC-AUC
    if y_prob is not None:
        auc = roc_auc_score(y_test, y_prob)
        print(f"Score ROC-AUC : {auc:.4f}")

    # 4. Matrice de Confusion
    print("\nMATRICE DE CONFUSION :")
    cm = confusion_matrix(y_test, y_pred)
    
    # Affichage textuel "propre"
    print(f"Vrais Négatifs (Restent) : {cm[0][0]:<5} | Faux Positifs (Erreur départ) : {cm[0][1]}")
    print(f"Faux Négatifs (Erreur reste): {cm[1][0]:<5} | Vrais Positifs (Partent)       : {cm[1][1]}")

    # 5. Affichage Plotly (Optionnel)
    if plot_plotly:
        fig = px.imshow(cm, 
                        text_auto=True, 
                        aspect="auto", 
                        labels=dict(x="Prédiction", y="Réalité", color="Nombre"),
                        x=['Reste (0)', 'Part (1)'],
                        y=['Reste (0)', 'Part (1)'],
                        title=f"Matrice de Confusion : {model_name}",
                        color_continuous_scale='Blues')
        fig.show()

print("--- Séparation Train/Test ---")
X_train, X_test, y_train, y_test = train_test_split(X_essential, y, test_size=0.2, random_state=42, stratify=y)

# 1. Dummy Classifier
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
evaluer_modele(dummy, X_test, y_test, "Dummy Classifier")

# 2. Régression Logistique
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)
evaluer_modele(log_reg, X_test, y_test, "Régression Logistique (Base)", plot_plotly=True)

# 3. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
evaluer_modele(rf, X_test, y_test, "Random Forest (Base)",plot_plotly=True)

--- Séparation Train/Test ---

ÉVALUATION : Dummy Classifier
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       247
           1       0.00      0.00      0.00        47

    accuracy                           0.84       294
   macro avg       0.42      0.50      0.46       294
weighted avg       0.71      0.84      0.77       294

Score ROC-AUC : 0.5000

MATRICE DE CONFUSION :
Vrais Négatifs (Restent) : 247   | Faux Positifs (Erreur départ) : 0
Faux Négatifs (Erreur reste): 47    | Vrais Positifs (Partent)       : 0

ÉVALUATION : Régression Logistique (Base)
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       247
           1       0.53      0.17      0.26        47

    accuracy                           0.84       294
   macro avg       0.70      0.57      0.59       294
weighted avg       0.81      0.84      0.81       294

Score ROC-AUC : 0.7739

MATRICE DE CONFUSION :
Vrais 


ÉVALUATION : Random Forest (Base)
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       247
           1       0.48      0.21      0.29        47

    accuracy                           0.84       294
   macro avg       0.67      0.58      0.60       294
weighted avg       0.80      0.84      0.81       294

Score ROC-AUC : 0.7626

MATRICE DE CONFUSION :
Vrais Négatifs (Restent) : 236   | Faux Positifs (Erreur départ) : 11
Faux Négatifs (Erreur reste): 37    | Vrais Positifs (Partent)       : 10


## 6. Optimisation (GridSearch + SMOTE)
Nous optimisons le Random Forest pour maximiser le rappel (Recall), car il est crucial de ne pas rater un démissionnaire.

In [5]:
print("--- Optimisation des Hyperparamètres ---")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Pipeline avec SMOTE pour gérer le déséquilibre
pipe_rf = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])

# Grille de recherche
param_grid_rf = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_leaf': [2, 4],
    'model__class_weight': ['balanced', None]
}

print("Recherche des meilleurs paramètres...")
grid_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=cv, scoring='recall', n_jobs=-1)
grid_rf.fit(X_train, y_train)

best_rf = grid_rf.best_estimator_
print(f"Meilleurs paramètres : {grid_rf.best_params_}")
evaluer_modele(best_rf, X_test, y_test, "Random Forest (Optimisé)")

--- Optimisation des Hyperparamètres ---
Recherche des meilleurs paramètres...
Meilleurs paramètres : {'model__class_weight': 'balanced', 'model__max_depth': 5, 'model__min_samples_leaf': 4, 'model__n_estimators': 50}

ÉVALUATION : Random Forest (Optimisé)
              precision    recall  f1-score   support

           0       0.89      0.82      0.85       247
           1       0.33      0.47      0.39        47

    accuracy                           0.76       294
   macro avg       0.61      0.64      0.62       294
weighted avg       0.80      0.76      0.78       294

Score ROC-AUC : 0.6839

MATRICE DE CONFUSION :
Vrais Négatifs (Restent) : 202   | Faux Positifs (Erreur départ) : 45
Faux Négatifs (Erreur reste): 25    | Vrais Positifs (Partent)       : 22


## 7. Optimisation Régression Logistique
Nous optimisons également la Régression Logistique, qui offre souvent une meilleure interprétabilité.

In [6]:
print("--- Optimisation Régression Logistique ---")

# Pipeline : Scaling -> SMOTE -> Modèle
pipe_log = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', LogisticRegression(max_iter=1000, random_state=42))
])

# Grille de paramètres
param_grid_log = {
    'model__C': [0.01, 0.1, 1, 5, 10],
    'model__class_weight': ['balanced', None],
    'smote__k_neighbors': [1, 3, 5, 7]
}

print("Recherche des meilleurs paramètres (LogReg)...")
grid_log = GridSearchCV(pipe_log, param_grid_log, cv=cv, scoring='recall', n_jobs=-1)
grid_log.fit(X_train, y_train)

best_log_reg = grid_log.best_estimator_
print(f"Meilleurs paramètres LogReg : {grid_log.best_params_}")
evaluer_modele(best_log_reg, X_test, y_test, "Régression Logistique (Optimisée)")

--- Optimisation Régression Logistique ---
Recherche des meilleurs paramètres (LogReg)...
Meilleurs paramètres LogReg : {'model__C': 0.1, 'model__class_weight': 'balanced', 'smote__k_neighbors': 1}

ÉVALUATION : Régression Logistique (Optimisée)
              precision    recall  f1-score   support

           0       0.92      0.73      0.81       247
           1       0.32      0.66      0.43        47

    accuracy                           0.72       294
   macro avg       0.62      0.69      0.62       294
weighted avg       0.82      0.72      0.75       294

Score ROC-AUC : 0.7697

MATRICE DE CONFUSION :
Vrais Négatifs (Restent) : 180   | Faux Positifs (Erreur départ) : 67
Faux Négatifs (Erreur reste): 16    | Vrais Positifs (Partent)       : 31


On avait pas fait un modèle parfait de base malheureusement, mais bon c'est celui qu'on a pour aujourd'hui.

## 8. Sauvegarde du Modèle
Nous sauvegardons le modèle **Régression Logistique** (qui a montré de meilleures performances) et la liste des features.

In [7]:
print("--- Sauvegarde des Artefacts ---")
output_dir = '..'

# Sauvegarde du modèle (Régression Logistique)
# Note: best_log_reg est un Pipeline (Scaler -> SMOTE -> LogReg)
joblib.dump(best_log_reg, f'{output_dir}/model.joblib')

# Sauvegarde des features
joblib.dump(X_essential.columns.tolist(), f'{output_dir}/features.joblib')

print(f"Modèle et features sauvegardés dans '{output_dir}'.")

--- Sauvegarde des Artefacts ---
Modèle et features sauvegardés dans '..'.
